In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from glob import glob
from time import time


device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import sys
sys.path.append('../')

from Inference import Inference as inf
from models import ViT, configs
from utils import dataloader

In [4]:
data_loader_training, data_loader_validate, data_loader_test = dataloader.return_dataloaders(batch_size = 1, 
                                                                                            in_channels = 4, 
                                                                                            lds_ks = 10,
                                                                                            lds_sigma = 8, 
                                                                                            dw_alpha = 3.9, 
                                                                                            betha  = 4, 
                                                                                            reweighting_method = 'lds', 
                                                                                            resmapling_status = False,
                                                                                            exp_name = 't1')

(13875, 33) | (8233, 33) | (12443, 33)


In [17]:
sample = next(iter(data_loader_test))   
print(sample["image"].shape, sample["mask"].shape)
print(sample["EmbList"])

torch.Size([1, 4, 16, 16, 15]) torch.Size([1, 1, 16, 16, 1])
[tensor([2]), tensor([3]), tensor([11]), tensor([5])]


In [5]:
config = configs.SRTR_Configs(img_size = 16, patch_size = 8, embed_dim = 1024, mlp_dim = 512, 
                    in_channels = 4, out_channels = 15, num_heads = 8, num_layers = 6, cond = None,
                    Attn_drop = 0.1, Proj_drop = 0.1, PostNorm = False, vis = True, 
                    kernel_size = [3, 3], dilation = [1, 1], stride  = [1, 1]
                    ).call()

In [6]:
model = ViT.ours(config).to(device)
# best_model_name = '/data2/hkaman/DiT/EXPs/EXP_vanilla_mse_01_0001_1024_8_6/best_model_vanilla_mse_01_0001_1024_8_6.pth'
# model.load_state_dict(torch.load(best_model_name))

In [24]:
pred = model(sample["image"].to(device), sample["EmbList"])

r2, mae, rmse, mape, _,_ = inf.regression_metrics(sample["mask"][0, 0, :, :, 0].detach().cpu().numpy(), pred[0, 0, :, :].detach().cpu().numpy())
scores = (r'$R^2={:.2f}$' + '\n' + r'MAE={:.2f}' + '\n' + r'$RMSE={:.2f}$' + '\n' + r'$MAPE={:.2f}$').format(
    r2, mae, rmse, mape)
print(scores)

$R^2=-1.17$
MAE=1.23
$RMSE=1.71$
$MAPE=0.15$


In [8]:
from models import engine
ImYiEst = engine.ImbYieldEst(model, 
                                lr = 0.01, 
                                wd = 0.0001, 
                                exp = 'vanilla_mse_01_0001_1024_8_6')

_ = ImYiEst.predict(model, data_loader_training, data_loader_validate, data_loader_test)

min = 1.797349214553833 | max = 13.181493759155273
min = 11.825191497802734 | max = 12.03119945526123
min = 4.343193054199219 | max = 13.181493759155273
min = 11.825191497802734 | max = 12.03119945526123
min = 4.343193054199219 | max = 13.181493759155273
min = 11.825191497802734 | max = 12.03119945526123
min = 9.351806640625 | max = 13.181493759155273
min = 11.825191497802734 | max = 12.03119945526123
min = 9.351806640625 | max = 13.181493759155273
min = 11.825191497802734 | max = 12.03119945526123
min = 9.351806640625 | max = 13.034149169921875
min = 11.825191497802734 | max = 12.03119945526123
min = 9.351806640625 | max = 13.034149169921875
min = 11.825191497802734 | max = 12.03119945526123
min = 9.435209274291992 | max = 13.034149169921875
min = 11.825191497802734 | max = 12.03119945526123
min = 9.510431289672852 | max = 13.034149169921875
min = 11.825191497802734 | max = 12.03119945526123
min = 9.777206420898438 | max = 13.034149169921875
min = 11.825191497802734 | max = 12.0311994

KeyboardInterrupt: 